# Batch-wise Structure Relaxation

In this tutorial, we show how to use the ``TorchStructureLBFGS``. It enables relaxation of structures in a batch-wise manner, i.e. it optimizes multiple structures in parallel. This is particularly useful, when many relatively similar structures (--> similar time until convergence) should be relaxed while requiring possibly short simulation time.

In [ ]:
import os
import torch
import random
import numpy as np
import matplotlib.pyplot as plt

import ase
from ase import Atoms
from ase.io import read, write

import schnetpack as spk
from schnetpack import properties
from schnetpack.interfaces.ase_interface import AtomsConverter
from schnetpack.interfaces.batchwise_optimization import ASEBatchwiseLBFGS
from schnetpack.interfaces.batchwise_optimization import BatchwiseCalculator, BatchwiseEnsembleCalculator

First, we load the force field model that provides the forces for the relaxation process. Furthermore, we define the atoms converter, which is used to convert ase Atoms objects to SchNetPack input. Eventually the calculator is initialized. The latter provides the necessary functionality to load a model and calculates forces and energy for the respective structures.

In [ ]:
model_path = "../../tests/testdata/md_ethanol.model"

# set device
device = torch.device("cpu")
dtype = torch.float32

# load model
model = torch.load(model_path, map_location=device)

# define neighbor list
cutoff = model.representation.cutoff.item()
nbh_list=spk.transform.MatScipyNeighborList(cutoff=cutoff)

# build calculator
calculator_ensemble = BatchwiseEnsembleCalculator(
    model=[model_path, model_path],
    neighbor_list=nbh_list,
    energy_key="energy",
    force_key="forces",
    energy_unit="kcal/mol",
    position_unit="Angstrom",
    device=device,
    dtype=dtype
    #converter=AtomsConverter,
    #transforms=None
    #additional_inputs=None,
    #auxiliary_output_modules=None,
    #ensemble_average_strategy=None,
)

calculator = BatchwiseCalculator(
    model=model_path,
    neighbor_list=nbh_list,
    energy_key="energy",
    force_key="forces",
    energy_unit="kcal/mol",
    position_unit="Angstrom",
    device=device,
    dtype=dtype
    #converter=AtomsConverter,
    #transforms=None
    #additional_inputs=None,
    #auxiliary_output_modules=None,
    #ensemble_average_strategy=None,
)

Subsequently, we load the batch of initial structures utilizing ASE (supports xyz, db and more).

In [ ]:
if not os.path.exists('howto_batchwise_relaxations_outputs'):
    os.makedirs('howto_batchwise_relaxations_outputs')
    
input_structure_file = "../../tests/testdata/md_ethanol.xyz"
random.seed(42)
batch_size = 30

ats = []
for mol_idx in range(batch_size):# load initial structure
    mol = read(input_structure_file)
    pos = mol.get_positions()
    # distort the structures and store them
    for n in range(pos.shape[0]):
        pos[n] = pos[n] * random.uniform(0.95,1.05)
    ats.append(Atoms(positions=pos, numbers=mol.get_atomic_numbers()))

For some systems it helps to fix the positions of certain atoms during the relaxation. This can be achieved by providing a mask of boolean entries to ``ASEBatchwiseLBFGS``. The mask is a list of $n_\text{atoms}$ entries, indicating atoms, which positions are fixed during the relaxation. Here, we do not fix any atoms. Hence, the mask only contains ``True``.

In [ ]:
# define structure mask for optimization (True for fixed, False for non-fixed)
n_atoms = len(ats[0].get_atomic_numbers())
single_structure_mask = [False for _ in range(n_atoms)]
# expand mask by number of input structures (fixed atoms are equivalent for all input structures)
mask = single_structure_mask * len(ats)

Finally, we run the optimization:

In [ ]:
# Initialize optimizer
optimizer = ASEBatchwiseLBFGS(
    calculator=calculator,
    atoms=ats,
    trajectory="./howto_batchwise_relaxations_outputs/relax_traj",
    verbose=True,
    log_every_step=False,
)

# run optimization
optimizer.run(fmax=0.0005, steps=1000)

In [ ]:
# Initialize optimizer
optimizer = ASEBatchwiseLBFGS(
    calculator=calculator_ensemble,
    atoms=ats,
    trajectory="./howto_batchwise_relaxations_outputs/relax_traj_ensemble",
    verbose=True,
    log_every_step=False,
)

# run optimization
optimizer.run(fmax=0.0005, steps=1000)

Optimzed structures (in the form of ASE `Atoms`) and properties can be obtained with the `get_relaxation_results` function.

In [ ]:
# get list of optimized structures and properties
opt_atoms, opt_props = optimizer.get_relaxation_results()

for oatoms in opt_atoms:
    print(oatoms.get_positions())
    
print(opt_props)

In [ ]:
# get all rmsds
rmsds = []
for traj_idx in range(batch_size):

    traj_file_1 = "./howto_batchwise_relaxations_outputs/relax_traj_ensemble_{}.xyz".format(traj_idx)
    traj_file_2 = "./howto_batchwise_relaxations_outputs/relax_traj_{}.xyz".format(traj_idx)

    traj_ats_1 = read(traj_file_1, index=":")
    traj_ats_2 = read(traj_file_2, index=":")

    for step_idx, (at_i, at_j) in enumerate(zip(traj_ats_1, traj_ats_2)):
        # rotate and translate to minimize rmsd
        ase.build.minimize_rotation_and_translation(at_i, at_j)

        # store reference and current structure in one atom object for visual comparison with "ase gui"
        pos_store = np.concatenate((at_i.get_positions(), at_j.get_positions()), axis=0)
        at_nums_store = np.concatenate((at_i.get_atomic_numbers(), at_j.get_atomic_numbers()), axis=0)
        at_store = Atoms(positions=pos_store, numbers=at_nums_store)
        write(
            os.path.join("./howto_batchwise_relaxations_outputs/comp.xyz"),
            at_store,
            format="extxyz",
            append=False if step_idx == 0 and traj_idx == 0 else True,
        )

        # calculate rmsd
        squared_distances = (at_i.get_positions() - at_j.get_positions()) ** 2
        rmsds.append(squared_distances.sum(1).mean())

print("RMSD_max = %.2fe-9 A" % (max(rmsds) * 1e9))

In [ ]:
fig = plt.figure()
plt.scatter(range(len(rmsds)), rmsds)
plt.show()